In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["backoffice"]

db.proveedor.insert_one({
    "cuit": "30-66060817-5",
    "razon_social": "Proveedores Tecnología SA",
    "activo": True,
    "habilitado": True
})

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
from pymongo import MongoClient

client = MongoClient("mongodb://mongo:27017")
db = client["backoffice"]

proveedores = db.proveedor.find()

with open("proveedores.txt", "w", encoding="utf-8") as f:
    for proveedor in proveedores:
        f.write(str(proveedor) + "\n")

In [2]:
from neo4j import GraphDatabase

# Conexión al servicio Bolt de Neo4j (en docker)
uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

# Función para insertar un proveedor
def insertar_proveedor(tx, cuit, razon_social, activo, habilitado):
    tx.run(
        """
        CREATE (p:Proveedor {
            cuit: $cuit,
            razon_social: $razon_social,
            activo: $activo,
            habilitado: $habilitado
        })
        """,
        cuit=cuit,
        razon_social=razon_social,
        activo=activo,
        habilitado=habilitado
    )

# Ejecutar la inserción
with driver.session() as session:
    session.execute_write(
        insertar_proveedor,
        "30-66060817-5",
        "Proveedores Tecnología SA",
        True,
        True
    )


ModuleNotFoundError: No module named 'neo4j'

In [1]:
from neo4j import GraphDatabase

uri = "bolt://neo4j:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "longerpass123"))

def obtener_proveedores(tx):
    result = tx.run("MATCH (p:Proveedor) RETURN p")
    return [record["p"] for record in result]

with driver.session() as session:
    proveedores = session.execute_read(obtener_proveedores)

for p in proveedores:
    print(dict(p))  # convierte el nodo a un diccionario para ver sus propiedades


ModuleNotFoundError: No module named 'neo4j'